In [6]:
train_dir = 'dogs/working/resized/train'
test_dir = 'dogs/working/resized/test'

In [7]:
from keras.preprocessing.image import ImageDataGenerator

# create a new generator
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory(train_dir, class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory(test_dir, class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

Found 16464 images belonging to 120 classes.
Found 4116 images belonging to 120 classes.


In [28]:
import time
start = time.time()
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

# build a sequential model
model = Sequential()
model.add(InputLayer(input_shape=(227, 227, 3)))

# 1st conv block
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=120, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=120, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fit on data for 30 epochs
model.fit_generator(train, epochs=30, validation_data=val)
end = time.time()
duration = end - start

/Users/teresa.dong/opt/anaconda3/envs/isye6740/lib/python3.8/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
129/129 [==============================] - 230s 2s/step - loss: 5.1024 - accuracy: 0.0062 - val_loss: 4.7905 - val_accuracy: 0.0095
Epoch 2/30
129/129 [==============================] - 242s 2s/step - loss: 4.7959 - accuracy: 0.0061 - val_loss: 4.7913 - val_accuracy: 0.0085
Epoch 3/30
129/129 [==============================] - 249s 2s/step - loss: 4.7974 - accuracy: 0.0039 - val_loss: 4.8762 - val_accuracy: 0.0109
Epoch 4/30
 25/129 [====>.........................] - ETA: 3:15 - loss: 4.8802 - accuracy: 0.0157

KeyboardInterrupt: 

In [13]:
loss, accuracy = model.evaluate(val)

print(loss,accuracy)

33/33 [==============================] - 12s 347ms/step - loss: 5.6066 - accuracy: 0.0532
5.606575012207031 0.053206998854875565


Transfer Learning

In [19]:
import time
import keras
from keras.applications.vgg16 import VGG16

# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [24]:
model3 = Sequential()
model3.add(InputLayer(input_shape=(227, 227, 3)))
model3.add(pretrained_model)
model3.add(GlobalAveragePooling2D())
model3.add(Dropout(0.5))
model3.add(Dense(120,activation='softmax'))

In [25]:
# compile the model
model3.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [26]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_3 (ModuleWrap (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 120)               61560     
Total params: 14,776,248
Trainable params: 14,776,248
Non-trainable params: 0
_________________________________________________________________


In [27]:
start = time.time()
#train model using features generated from VGG16 model
model3.fit_generator(train, epochs=30, validation_data=val)
end=time.time()

/Users/teresa.dong/opt/anaconda3/envs/isye6740/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
 29/129 [=====>........................] - ETA: 1:01:06 - loss: 29.2773 - accuracy: 0.0046

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
import time
start = time.time()




model2 = Sequential()
model2.add(Flatten(input_shape=(227,227,3)))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(120, activation='softmax'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit_generator(train, epochs=30, validation_data=val)
end = time.time()
duration_cnn_vgg16 = end-start
print(duration_cnn_vgg16)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 154587)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               15458800  
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 120)               12120     
Total params: 15,471,320
Trainable params: 15,471,120
Non-trainable params: 200
_________________________________________________________________
Epoch 1/30


/Users/teresa.dong/opt/anaconda3/envs/isye6740/lib/python3.8/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


129/129 [==============================] - 38s 292ms/step - loss: 5.0029 - accuracy: 0.0050 - val_loss: 4.7842 - val_accuracy: 0.0114
Epoch 2/30
129/129 [==============================] - 31s 237ms/step - loss: 4.8556 - accuracy: 0.0079 - val_loss: 4.7705 - val_accuracy: 0.0117
Epoch 3/30
129/129 [==============================] - 30s 233ms/step - loss: 4.8141 - accuracy: 0.0096 - val_loss: 4.7452 - val_accuracy: 0.0136
Epoch 4/30
129/129 [==============================] - 30s 230ms/step - loss: 4.7863 - accuracy: 0.0121 - val_loss: 4.7433 - val_accuracy: 0.0192
Epoch 5/30
129/129 [==============================] - 30s 230ms/step - loss: 4.7838 - accuracy: 0.0118 - val_loss: 4.7389 - val_accuracy: 0.0216
Epoch 6/30
129/129 [==============================] - 30s 229ms/step - loss: 4.7689 - accuracy: 0.0106 - val_loss: 4.7231 - val_accuracy: 0.0190
Epoch 7/30
129/129 [==============================] - 30s 230ms/step - loss: 4.7602 - accuracy: 0.0132 - val_loss: 4.7130 - val_accuracy: 0.0

In [12]:
loss, accuracy = model2.evaluate(val)

print(loss,accuracy)

33/33 [==============================] - 6s 185ms/step - loss: 4.6803 - accuracy: 0.0267
4.680278301239014 0.026724975556135178
